In [1]:
pip install deepctr

     |████████████████████████████████| 112kB 5.7MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Tobigs/컨퍼런스_와인추천/

Mounted at /content/drive
/content/drive/My Drive/Tobigs/컨퍼런스_와인추천


In [3]:
import pandas as pd
import numpy as np
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tf.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, roc_auc_score

In [4]:
train_df = pd.read_json('Data 최종본/train_all_meta_v2.json')
test_df = pd.read_json('Data 최종본/test_all_meta_v2.json')

feature : userID, food, grapes, region_id + 연속형 변수

In [5]:
X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count','userID','food','grapes','region_id']]
y_train = train_df['like']
X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count','userID','food','grapes','region_id']]
y_test = test_df['like']

In [6]:
# food 리스트 -> str
food_str = []
for i in range(len(X_train['food'])):
    if X_train['food'][i]:food_str.append(' '.join(X_train['food'][i]))
    else:food_str.append(' ')
X_train.food = pd.Series(food_str, name='food')

food_str = []
for i in range(len(X_test['food'])):
    if X_test['food'][i]:food_str.append(' '.join(X_test['food'][i]))
    else:food_str.append(' ')
X_test.food = pd.Series(food_str, name='food')

# grapes 리스트 -> str
grapes_str = []
for i in range(len(X_train['grapes'])):
    if X_train['grapes'][i]:grapes_str.append(' '.join(X_train['grapes'][i]))
    else:grapes_str.append(' ')
X_train.grapes = pd.Series(grapes_str, name='grapes')

grapes_str = []
for i in range(len(X_test['grapes'])):
    if X_test['grapes'][i]:grapes_str.append(' '.join(X_test['grapes'][i]))
    else:grapes_str.append(' ')
X_test.grapes = pd.Series(grapes_str, name='grapes')

# userID, region_id -> str
X_train.userID = X_train.userID.astype(str)
X_train.region_id = X_train.region_id.astype(str)

X_test.userID = X_test.userID.astype(str)
X_test.region_id = X_test.region_id.astype(str)

In [7]:
sparse_features = ['userID', 'food', 'grapes', 'region_id']

dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol',
       'winery_ratings_count', 'winery_ratings_average', 'user_follower_count',
       'user_following_count', 'user_rating_count']

for feat in sparse_features:
        lbe = LabelEncoder()
        all = pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates() # train, test 전체를 묶어서 LabelEncoder fit
        lbe = lbe.fit(all)
        X_train[feat] = lbe.transform(X_train[feat])
        X_test[feat] = lbe.transform(X_test[feat])

mms = MinMaxScaler(feature_range=(0, 1))
X_train[dense_features] = mms.fit_transform(X_train[dense_features])
X_test[dense_features] = mms.transform(X_test[dense_features])

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=X_train[feat].nunique(),embedding_dim=4)
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 결측치 mean으로 처리
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

train_model_input = {name:X_train[name] for name in feature_names}
test_model_input = {name:X_test[name] for name in feature_names}

In [8]:
# parameters
BATCH_SIZE = 256
EPOCHS = 100
# early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [9]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])

history = model.fit(train_model_input, y_train.values,
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=2, validation_split=0.2)
y_pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
print("test AUC", round(roc_auc_score(y_test.values, y_pred), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, y_pred.round()), 4))

Epoch 1/100
2386/2386 - 18s - loss: 0.5707 - binary_crossentropy: 0.5701 - val_loss: 0.6516 - val_binary_crossentropy: 0.6506
Epoch 2/100
2386/2386 - 14s - loss: 0.5421 - binary_crossentropy: 0.5405 - val_loss: 0.6638 - val_binary_crossentropy: 0.6618
Epoch 3/100
2386/2386 - 14s - loss: 0.5358 - binary_crossentropy: 0.5333 - val_loss: 0.6615 - val_binary_crossentropy: 0.6586
Epoch 4/100
2386/2386 - 13s - loss: 0.5297 - binary_crossentropy: 0.5265 - val_loss: 0.6549 - val_binary_crossentropy: 0.6514
Epoch 5/100
2386/2386 - 13s - loss: 0.5256 - binary_crossentropy: 0.5218 - val_loss: 0.6499 - val_binary_crossentropy: 0.6459
Epoch 6/100
2386/2386 - 14s - loss: 0.5225 - binary_crossentropy: 0.5181 - val_loss: 0.6457 - val_binary_crossentropy: 0.6411
Epoch 7/100
2386/2386 - 14s - loss: 0.5201 - binary_crossentropy: 0.5152 - val_loss: 0.6504 - val_binary_crossentropy: 0.6453
Epoch 8/100
2386/2386 - 14s - loss: 0.5181 - binary_crossentropy: 0.5126 - val_loss: 0.6560 - val_binary_crossentropy: